## **1. Load library**

In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors

## **2. Load data**

In [2]:
raw = pd.read_csv('./models/solubility/delaney_solubility.csv')
df_work = raw
df_work

,Compound ID,measured log(solubility:mol/L),ESOL predicted log(solubility:mol/L),SMILES
0,"1,1,1,2-Tetrachloroethane",-2.180,-2.794,ClCC(Cl)(Cl)Cl
1,"1,1,1-Trichloroethane",-2.000,-2.232,CC(Cl)(Cl)Cl
2,"1,1,2,2-Tetrachloroethane",-1.740,-2.549,ClC(Cl)C(Cl)Cl
3,"1,1,2-Trichloroethane",-1.480,-1.961,ClCC(Cl)Cl
4,"1,1,2-Trichlorotrifluoroethane",-3.040,-3.077,FC(F)(Cl)C(F)(Cl)Cl
...,...,...,...,...
1139,vamidothion,1.144,-1.446,CNC(=O)C(C)SCCSP(=O)(OC)(OC)
1140,Vinclozolin,-4.925,-4.377,CC1(OC(=O)N(C1=O)c2cc(Cl)cc(Cl)c2)C=C
1141,Warfarin,-3.893,-3.913,CC(=O)CC(c1ccccc1)c3c(O)c2ccccc2oc3=O
1142,Xipamide,-3.790,-3.642,Cc1cccc(C)c1NC(=O)c2cc(c(Cl)cc2O)S(N)(=O)=O


## **3. Calculate molecular descriptors in rdkit**

### **3.1. Calculate molecular descriptors**

To predict **LogS** (log of the aqueous solubility), the study by Delaney makes use of 4 molecular descriptors:
1. **cLogP** *(Octanol-water partition coefficient)*
2. **MW** *(Molecular weight)*
3. **RB** *(Number of rotatable bonds)*
4. **AP** *(Aromatic proportion = number of aromatic atoms / total number of heavy atoms)*

In [3]:
def AromaticProportion(mol):
    aromatic_atoms = [mol.GetAtomWithIdx(i).GetIsAromatic() for i in range(mol.GetNumAtoms())]
    AA_count = []
    
    for elem in aromatic_atoms:
        if elem==True:
            AA_count.append(1)
    AromaticAtom = sum(AA_count)
    
    HeavyAtom = Descriptors.HeavyAtomCount(mol)
    AR = AromaticAtom/HeavyAtom
    
    return AR   
    

def getDescriptors(mol):

    desc_LogP = Descriptors.MolLogP(mol)
    desc_MW = Descriptors.MolWt(mol)
    desc_RotatableBonds = Descriptors.NumRotatableBonds(mol)
    desc_AromaticProportion = AromaticProportion(mol)
    
    descriptors = np.array([desc_LogP,
                    desc_MW,
                    desc_RotatableBonds,
                    desc_AromaticProportion])
    
    return descriptors

### **3.2. Create dataframe with the descriptors**

In [4]:
mol_list = []
desc_values = []

for element in df_work.SMILES:
    mol_list.append(Chem.MolFromSmiles(element))
    
for mol_elem in mol_list:
    descriptors = getDescriptors(mol_elem)
    desc_values.append(descriptors)
    
columnNames=['LogP', 'Mol Wt', 'Num Rotatable Bonds', 'Aromatic Proportion']   
descriptors = pd.DataFrame(data=desc_values, columns=columnNames)
    
descriptors

,LogP,Mol Wt,Num Rotatable Bonds,Aromatic Proportion
0,2.59540,167.850,0.0,0.000000
1,2.37650,133.405,0.0,0.000000
2,2.59380,167.850,1.0,0.000000
3,2.02890,133.405,1.0,0.000000
4,2.91890,187.375,1.0,0.000000
...,...,...,...,...
1139,1.98820,287.343,8.0,0.000000
1140,3.42130,286.114,2.0,0.333333
1141,3.60960,308.333,4.0,0.695652
1142,2.56214,354.815,3.0,0.521739


### **3.3. Create dataframe with the target descriptor**

In [5]:
target = df_work.iloc[:,1]
target = target.rename('logS')
target

0      -2.180
1      -2.000
2      -1.740
3      -1.480
4      -3.040
        ...  
1139    1.144
1140   -4.925
1141   -3.893
1142   -3.790
1143   -2.581
Name: logS, Length: 1144, dtype: float64

## **3.4. Combine both dataset into same dataframe**

In [6]:
dataset = pd.concat([descriptors, target], axis=1)
dataset

,LogP,Mol Wt,Num Rotatable Bonds,Aromatic Proportion,logS
0,2.59540,167.850,0.0,0.000000,-2.180
1,2.37650,133.405,0.0,0.000000,-2.000
2,2.59380,167.850,1.0,0.000000,-1.740
3,2.02890,133.405,1.0,0.000000,-1.480
4,2.91890,187.375,1.0,0.000000,-3.040
...,...,...,...,...,...
1139,1.98820,287.343,8.0,0.000000,1.144
1140,3.42130,286.114,2.0,0.333333,-4.925
1141,3.60960,308.333,4.0,0.695652,-3.893
1142,2.56214,354.815,3.0,0.521739,-3.790


## **4. Save**

In [7]:
dataset.to_csv('./models/solubility/delaney_solubility_descriptors.csv', index=False)